# Doctor Type ML Automation
Amy Jin

July 10th, 2018

In [1]:
# Load libraries
import pandas as pd
import seaborn as sns
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import RandomizedPCA
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from collections import Counter
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
# global import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import time
from imblearn.over_sampling import RandomOverSampler
# Connect to Parenthood server
import mysql.connector
import sshtunnel
import pureyaml


In [32]:
def GetDataFromParenthood(db_name, table_name):
    
    # handle path
    project_dir = !pwd  # dir of current script/notebook file
    config_file = open(project_dir[0] + "/db.yaml")
    config = pureyaml.load(config_file.read())
    config_file2 = open(project_dir[0] + "/db2.yaml")
    config2 = pureyaml.load(config_file2.read())

    # argument dictionary for sshtunnel
    ssh_config = {
        'ssh_address_or_host': ('parenthood.set.care', 22),
        'ssh_username':        config['ssh_username'],
        'ssh_password':        config['ssh_password'],
        'remote_bind_address': ('127.0.0.1', 3306),
        'local_bind_address':  ('0.0.0.0', 3333),
    }

    # argument dictionary for mysql.connector
    mysql_config = {
        'user':     config['mysql_user'],
        'password': config['mysql_passwd'],
        'host':     config['mysql_host'],
        'database': 'patch',
        'port':     3333,
    }  
    
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # Use `DISTINCT` to reduce the calculation load
        query = ('''
            SELECT *
            FROM {db}.{t1} AS A;
        '''.format(db = db_name, t1 = table_name))

        cur.execute(query)
        rows = list(sum(cur.fetchall(), ()))
        #print rows, len(rows)
        #df = pd.DataFrame(rows)
        #print df
        #print df.shape
        
        # Get columans 
        colnames = ['npi', 'Y_is_oncologist', 'Certification', 'patient_count']
        for i in range(1, 11):  # do the following 10 times:
            colnames.append('X' + str(i))

        # Transform a long list to a (n, 14) dataframe
        # -1 simply means that it is an unknown dimension and we want numpy to figure it out.
        df_reformat =  pd.DataFrame(np.array(rows).reshape(-1, 14), columns=colnames)
        return df_reformat
            
        cur.close()
        connection.close()

In [33]:
tic = time.clock()
data = GetDataFromParenthood('ml_provider_type', 'final_table')
toc = time.clock()
toc - tic

2738.5135969999997

In [2]:
data = pd.read_csv("/Users/yuejin/Documents/AmyJin_2018/Work/Test/2018_03/Doctor_Type_ML/final_table2.csv")

In [3]:
data.head(3)

,npi,Y_is_oncologist,Certification,patient_count,X_96413,X_96367,X_96361,X_80053,X_G0008,X_83615,X_82728,X_83550,X_J3490,X_Q2037
0,1003000134,0,Dermatopathology,4686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1003000407,0,Family Medicine/OMT,493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1003000480,0,Surgery,147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
print data.dtypes

npi                  int64
Y_is_oncologist      int64
Certification       object
patient_count        int64
X_96413            float64
X_96367            float64
X_96361            float64
X_80053            float64
X_G0008            float64
X_83615            float64
X_82728            float64
X_83550            float64
X_J3490            float64
X_Q2037            float64
dtype: object


In [35]:
def Model_Logistic(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    ros = RandomOverSampler(random_state=0)
    X_resampled, y_resampled = ros.fit_sample(X_train, y_train)
    
    logisticRegr = LogisticRegression(random_state=0)
    X_resampled = pd.DataFrame(X_resampled)
    logisticRegr.fit(X_resampled.iloc[:,[0,1,2,4,5,9]], y_resampled)
    y_data_pred = logisticRegr.predict(X_data.iloc[:,[0,1,2,4,5,9]])
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for Logistic Regression with 6 features is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [36]:
logist_pred = Model_Logistic(data)
print logist_pred

Confusion Matrix for Logistic Regression with 6 features is:
[[183399   2005]
 [   380    209]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [42]:
def Model_Tree(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
        
    ros = RandomOverSampler(random_state=0)
    X_resampled, y_resampled = ros.fit_sample(X_train, y_train)
    
    clf_gini_3 = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
    clf_gini_3.fit(X_resampled, y_resampled)
    y_data_pred = clf_gini_3.predict(X_data)

    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for Decision Tree with Gini Index is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [43]:
tree_pred = Model_Tree(data)
print tree_pred

Confusion Matrix for Decision Tree with Gini Index is:
[[185363     41]
 [   401    188]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [44]:
def Model_RF(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]

    clf_RF = RandomForestClassifier(n_estimators=100)
    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf_RF.fit(X_train,y_train)

    y_data_pred=clf_RF.predict(X_data)

    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for Random Forest with Gini Index is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [45]:
RF_pred = Model_RF(data)
print RF_pred

Confusion Matrix for Random Forest with Gini Index is:
[[185394     10]
 [   407    182]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [47]:
def Model_NaiveBayes(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]

    GNB = GaussianNB()
    GNB.fit(X_train, y_train)
    y_data_pred = GNB.predict(X_data)
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for Naive Bayes is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [48]:
NB_pred = Model_NaiveBayes(data)
print NB_pred

Confusion Matrix for Naive Bayes is:
[[182477   2927]
 [   383    206]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [57]:
def Model_SVM_sigmoid(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    pca = PCA(n_components=2)
    pca.fit(X_train)
    X_pca = pca.transform(X_train)
    train_2d = pca.transform(X_train)
    data_2d = pca.transform(X_data)
    
    pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
    sigmoid_SVM = SVC(kernel='sigmoid')
    sigmoid_SVM.fit(train_2d, y_train)
    y_data_pred = sigmoid_SVM.predict(data_2d)
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for SVM with sigmoid kernel is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [58]:
SVM_sigmoid_pred = Model_SVM_sigmoid(data)
print SVM_sigmoid_pred

Confusion Matrix for SVM with sigmoid kernel is:
[[185404      0]
 [   589      0]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [61]:
def Model_SVM_linear(data, C):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    pca = PCA(n_components=2)
    pca.fit(X_train)
    X_pca = pca.transform(X_train)
    train_2d = pca.transform(X_train)
    data_2d = pca.transform(X_data)
    
    pca = RandomizedPCA(n_components=150, whiten=True, random_state=42)
    linear_SVM = SVC(kernel='linear', class_weight='balanced', C = 1)
    linear_SVM.fit(train_2d, y_train)
    y_data_pred = linear_SVM.predict(data_2d)


    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for SVM with linear kernel is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [62]:
SVM_linear_pred = Model_SVM_linear(data, 1)
print SVM_linear_pred

Confusion Matrix for SVM with linear kernel is:
[[180569   4835]
 [   481    108]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [63]:
def Model_kNN_linear(data, n_neighbors):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    knn = KNeighborsClassifier(n_neighbors=3)
    # fitting the model
    knn.fit(X_train, y_train)
    # predict the response
    y_data_pred = knn.predict(X_data)
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for kNN is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [64]:
kNN_pred = Model_kNN_linear(data, 10)
print kNN_pred

Confusion Matrix for kNN is:
[[185373     31]
 [   532     57]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [65]:
def Model_LDA_linear(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]

    lda = LinearDiscriminantAnalysis()
    model_lda = lda.fit(X_train, y_train)
    y_data_pred=model_lda.predict(X_data)

    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for LDA is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [66]:
LDA_pred = Model_LDA_linear(data)
print LDA_pred

Confusion Matrix for LDA is:
[[184042   1362]
 [   395    194]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [70]:
def Model_QDA_linear(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    qda = QuadraticDiscriminantAnalysis()
    model_qda = qda.fit(X_train, y_train)
    y_data_pred=model_qda.predict(X_data)

    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for QDA is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [71]:
QDA_pred = Model_QDA_linear(data)
print QDA_pred

Confusion Matrix for QDA is:
[[182389   3015]
 [   379    210]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [72]:
def Model_GBM_linear(data, n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    
    model_BGM= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
    # Train the model using the training sets and check score
    model_BGM.fit(X_train, y_train)
    y_data_pred = model_BGM.predict(X_data)
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for GBM is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [73]:
GBM_pred = Model_GBM_linear(data)
print GBM_pred

Confusion Matrix for GBM is:
[[184253   1151]
 [   399    190]]
[u'0' u'0' u'0' ... u'0' u'0' u'0']


In [91]:
def Model_XGBoost_linear(data):
    
    train, test = train_test_split(data, test_size=0.2, random_state = 100)
    X_train = train.iloc[:,4:14]
    y_train = train.iloc[:,1]
    X_data = data.iloc[:,4:14]
    y_data = data.iloc[:,1]
    #print X_train.dtypes
    
    model_XGB = XGBClassifier()
    model_XGB.fit(X_train, y_train)
    y_data_pred = model_XGB.predict(X_data)
    
    confusion_mtx = confusion_matrix(y_data, y_data_pred)
    print "Confusion Matrix for XGBoost is:"
    print(confusion_mtx)
    return y_data_pred
    #return confusion_mtx

In [92]:
XGBoost_pred = Model_XGBoost_linear(data)
print XGBoost_pred

Confusion Matrix for XGBoost is:
[[185397      7]
 [   573     16]]
[0 0 0 ... 0 0 0]


In [9]:
def UploadPredictionsToParenthood(db_name, db_table_name, table_to_upload):

    # handle path
    project_dir = !pwd  # dir of current script/notebook file
    config_file = open(project_dir[0] + "/db.yaml")
    config = pureyaml.load(config_file.read())
    config_file2 = open(project_dir[0] + "/db2.yaml")
    config2 = pureyaml.load(config_file2.read())

    # argument dictionary for sshtunnel
    ssh_config = {
        'ssh_address_or_host': ('parenthood.set.care', 22),
        'ssh_username':        config['ssh_username'],
        'ssh_password':        config['ssh_password'],
        'remote_bind_address': ('127.0.0.1', 3306),
        'local_bind_address':  ('0.0.0.0', 3333),
    }

    # argument dictionary for mysql.connector
    mysql_config = {
        'user':     config['mysql_user'],
        'password': config['mysql_passwd'],
        'host':     config['mysql_host'],
        'database': 'patch',
        'port':     3333,
    }       
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:

        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # Import dataframe into MySQL
        import sqlalchemy
        database_username = config2['database_username']
        database_password = config2['database_password']
        database_ip       = config2['database_ip']
        database_name     = db_name
    
        database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                       format(database_username, database_password, 
                                                              database_ip, database_name))
        table_to_upload.to_sql(con=database_connection, name= db_table_name, if_exists='replace')
        print "Table has been uploaded successfully!"
            
        cur.close()
        connection.close()


In [10]:
UploadPredictionsToParenthood('ml_provider_type','test_table_2', test_df_2)

Table has been uploaded successfully!
